# PDF Downloader

In [51]:
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time

In [4]:
dtype_dict = {'code': str, 'name': str, 'year': int, 'link': str}
df = pd.read_csv('./Data/annual_report_cleaned.csv', dtype=dtype_dict)
df.head()

,code,year,name,link
0,603019,2024,中科曙光,http://www.cninfo.com.cn/new/disclosure/detail...
1,872808,2024,曙光数创,http://www.cninfo.com.cn/new/disclosure/detail...
2,300353,2024,东土科技,http://www.cninfo.com.cn/new/disclosure/detail...
3,000063,2024,中兴通讯,http://www.cninfo.com.cn/new/disclosure/detail...
4,688041,2024,海光信息,http://www.cninfo.com.cn/new/disclosure/detail...


For each link: 
1. Access url and download PDF file
2. Append file local address to the Dataframe (Change your web browser's default downloading directory to your working directory)

Objective Data Structure:
- **code**
- **year**
- name
- local file path

In [65]:
driver_path = r"C:\Program Files\Google\chromedriver-win64\chromedriver.exe"  # your driver path
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
)

In [79]:
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
driver.maximize_window()

In [80]:
for i in range(20,50): # range(len(df))
    link = df.loc[i, 'link']
    driver.get(link)
    time.sleep(2)

    # retrieve file name information
    name = df.loc[i, 'name']
    name_ele = driver.find_element(By.XPATH, '//div[@class="title"]')
    name2 = name_ele.text

    # format the file path
    local_path = f'./Data/PDFs/{name}：{name2}.PDF'
    # append the file path to the dataframe
    df.loc[i, 'local_path'] = local_path

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[1]/div[3]/div[1]/button'))
    )

    # Click the download button
    button.click()
    time.sleep(0.5)
    print(f'Downloaded {i+1}')

# driver.quit()
print('All files downloaded')


Downloaded 21
Downloaded 22
Downloaded 23
Downloaded 24
Downloaded 25
Downloaded 26
Downloaded 27
Downloaded 28
Downloaded 29
Downloaded 30
Downloaded 31
Downloaded 32
Downloaded 33
Downloaded 34
Downloaded 35
Downloaded 36
Downloaded 37
Downloaded 38
Downloaded 39
Downloaded 40
Downloaded 41
Downloaded 42
Downloaded 43
Downloaded 44
Downloaded 45
Downloaded 46
Downloaded 47
Downloaded 48
Downloaded 49
Downloaded 50
All files downloaded


In [82]:
df['local_path'] = df['local_path'].str.replace('*', '_')

In [84]:
df = df.drop(columns='link')
df.to_csv('./Data/annual_report_with_path.csv', index=False)